In [1]:
import sys
sys.path.append('..')
from nocola_to_df import get_df
from datasets import Dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from models import SBERT_MODEL
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from constants import label_mapping

In [5]:
train_df = get_df(split="train")
dev_df = get_df(split="dev")
test_df = get_df(split="test")

train_df["label"] = train_df["label"].map(label_mapping)
test_df["label"] = test_df["label"].map(label_mapping)
dev_df["label"] = dev_df["label"].map(label_mapping)

def get_reduced_dataframe(df, min_samples=8):
    # proportion of each label, based on the minimum label
    proportion = df.label.value_counts() / min_samples
    # normalize it, so the lowest value is 1:
    proportion = proportion  / proportion.min()
    # multiply each by 8 (so that we ensure at least 8 samples)
    proportion = proportion * min_samples
    proportion = proportion.apply(round)
    proportion = proportion.to_dict()
    print(proportion)
    
    subset_dfs = []
    for label, desired_count in proportion.items():
        label_df = df[df['label'] == label]
        subset_dfs.append(label_df.sample(n=desired_count, random_state=42))
    subset = pd.concat(subset_dfs, axis=0).reset_index(drop=True)
    return subset

subset_train = get_reduced_dataframe(train_df, min_samples=8)
subset_eval = get_reduced_dataframe(dev_df, min_samples=3)
subset_test = get_reduced_dataframe(test_df, min_samples=3)

dataset = Dataset.from_pandas(subset_train)
dataset_test = Dataset.from_pandas(subset_test)
dataset_eval = Dataset.from_pandas(subset_eval)

{0.0: 90, 2.0: 74, 6.0: 73, 4.0: 62, 3.0: 59, 5.0: 49, 9.0: 28, 10.0: 10, 1.0: 8}
{0.0: 41, 2.0: 32, 6.0: 31, 3.0: 27, 4.0: 25, 5.0: 21, 9.0: 12, 10.0: 4, 1.0: 3}
{0.0: 41, 6.0: 34, 2.0: 31, 3.0: 29, 4.0: 27, 5.0: 19, 9.0: 13, 10.0: 5, 1.0: 3}


In [7]:
num_classes = len(dataset.unique("label"))

sfit_model = SetFitModel.from_pretrained(
    SBERT_MODEL,
    use_differentiable_head=True,
    head_params={"out_features": num_classes},
)
#Note: If you use the differentiable SetFitHead classifier head, it will automatically use BCEWithLogitsLoss for training.
# The prediction involves a sigmoid after which probabilities are rounded to 1 or 0.
# Furthermore, the "one-vs-rest" and "multi-output" multi-target strategies are equivalent for the differentiable SetFitHead.
# Create trainer
sfit_model.to("mps")
trainer = SetFitTrainer(
    model=sfit_model,
    train_dataset=dataset,
    eval_dataset=dataset_eval,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1, # Number of epochs to use for contrastive learning
)
trainer.freeze()
trainer.train()
trainer.unfreeze(keep_body_frozen=True)
trainer.train(
    num_epochs=25,
    batch_size=16,
    body_learning_rate=1e-5,  # LR of body
    learning_rate=1e-2,  # LR of head
    l2_weight=0.0
)
metrics = trainer.evaluate()
print(metrics)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 18120
  Num epochs = 1
  Total optimization steps = 1133
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1133 [00:00<?, ?it/s]

The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 75.


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

***** Running evaluation *****


{'accuracy': 0.15816326530612246}


In [9]:
model = trainer.model
model

SetFitModel()

In [17]:
dataset_test[-10:]

{'text': ['De nye forbilder er nå blitt popstjerner, "Spidermann", "Pokemón" og så videre, og foreldrene, som er opptatt med seg selv, ikke er så viktige lenger.',
  'Hvis man liker å være isolert, det passer bedre å bo på fjellet eller i skogen, men hvis man trives med mennesker, er det mye bedre å bo i byen.',
  'det var min mening.',
  'Jeg Håper at jeg klarer denne testen, og kanskje Bergenstesten, for å ha bedre sjanse til å komme inn.',
  'Det er viktig at man finner på en aktivitet, for da får man mulighet til bli kjent med andre. Man kan alltid Lære noe positivt fra dem.',
  'Et annet spørsmål er organdonasjon fra dyr. her er det nok mange som igjen vil se det som forkastelig å "blande" dyr med mennesker.',
  'I mitt hjemland, Nederland, snakker de fleste engelsk, sånn at man ikke føler seg helt isolert i begynnelsen, men over lengre tid lønner det seg å snakke Nederlandsk for å komme seg med i det sosiale livet.',
  'For eksempel kan de bruke tiden på etterutdanning, kunst, mu

In [ ]:
decoding = {v:k for k,v in label_mapping.items()}

In [27]:
subset_test.label.value_counts()

label
0.0     41
6.0     34
2.0     31
3.0     29
4.0     27
5.0     19
9.0     13
10.0     5
1.0      3
Name: count, dtype: int64

In [32]:
subset_test.label.value_counts()

label
0.0     41
6.0     34
2.0     31
3.0     29
4.0     27
5.0     19
9.0     13
10.0     5
1.0      3
Name: count, dtype: int64

In [33]:
subset_test.head()

,text,label
0,Han kan lage middag til familien og viser at m...,0.0
1,"Det aller viktigste for å ikke bli for tykk, e...",0.0
2,Mannen min og jeg kan kjøre bare noen minutter...,0.0
3,"Noen var lei av å være hjemme, og de var depri...",0.0
4,"Livet går videre, og vi må rett og slett gjøre...",0.0


In [50]:
from tqdm import tqdm
# iterate the test set, and get the predictions:
truths = []
preds = []
for text, label in tqdm(zip(subset_test.text, subset_test.label)):
    pred = model.predict(text).item()
    pred = float(pred)
    pred = decoding[pred]
    truth = decoding[label]
    print(text)
    print(f"Truth: {truth}, Pred: {pred}")
    print()
    truths.append(truth)
    preds.append(pred)

9it [00:00, 40.14it/s]

Han kan lage middag til familien og viser at matlaging ikke bare er for kvinner.
Truth: F, Pred: M

Det aller viktigste for å ikke bli for tykk, er vel en sunt kosthold, der frukt og grønnsaker spiller en stor rolle.
Truth: F, Pred: PUNC

Mannen min og jeg kan kjøre bare noen minutter og være ute i natur.
Truth: F, Pred: M

Noen var lei av å være hjemme, og de var deprimerte, fordi de ikke hadde sosiale kontakt med andre voksne.
Truth: F, Pred: M

Livet går videre, og vi må rett og slett gjøre noe for å bevare våre miljø som skal gitt videre til våre neste generasjoner.
Truth: F, Pred: W

Hos oss er det ganske ensomme av og til.
Truth: F, Pred: M

Det var ikke lett å bo i Norge frem til og med for 80 år siden, og det tror jeg er hovedgrunnen til hvorfor nordmenn er rolig og at de alltid vil hjelpe hvis du spør om hjelp.
Truth: F, Pred: M

Dårlige vaner hjemme kunne følge med en til et annet sted, men nye vaner burde man ta vare på, som for eksempel hvordan man gjør ting i dagliglivet o

14it [00:00, 38.31it/s]

Hvis du handle på Internett, var det veldig farlig, fordi andre kunne gå inn i dine betalingsmidler og bruke dine penger.
Truth: F, Pred: ORT

Det er veldig lett å gjøre det hvis de trenger litt. Etter hvert ble behov for varer større og større.
Truth: F, Pred: ORT

Flomen rammer disse landene hardt.
Truth: F, Pred: W

Jeg synes at hvis man ikke vil bli forstyrret, må man også sørge for at andre ikke bli forstyrret av en.
Truth: F, Pred: R

Hvordan er været i Kosovo om vinter og om sommeren?
Truth: F, Pred: R

Det finnes mange likheter og ulikheter mellom nærmiljø i Norge og i Ukraina.
Truth: F, Pred: PUNC

Barna får muligheter til å få den beste utdannelse og utviklingen fordi de kan studere ved private skoler.
Truth: F, Pred: R

Hun er den moderne forfatter og det er veldig interessant å lese bøkene hennes.
Truth: F, Pred: F

Jeg har lest en bok en gang.
Truth: F, Pred: W



24it [00:00, 42.20it/s]

...

Men jeg synes at det viktigste er det samarbeidet mellom foreldre og skole.
Truth: R, Pred: PUNC

Tid og energi som ellers kunne ha blitt brukt på en mer konstruktiv måte til å bygge opp et godt forhold til barna, er brukt dårlig utnyttet.
Truth: R, Pred: W

Vi kan få ringe etter hjelp og så videre.
Truth: R, Pred: M

Spesielt i de samfunnene hvor kvinner har langt fra de samme mulighetene til utdanning, ser vi ganske stor forskjell mellom de yrkene menn er inn i og de yrkene kvinner er i.
Truth: R, Pred: M

Man kan også se r på det på en annen måte.
Truth: R, Pred: F

Alltid ringer det mobiltelefoner når man sitter på kino, teater eller konsert er, som er steder hvor jeg synes at det ikke er akseptabelt at man ikke slår av mobiltelefonen.
Truth: R, Pred: F

Jeg håper at jeg skal fortsette en dag å studere dette yrket i Norge, og endelig få r jobben som jeg liker best.
Truth: R, Pred: M

Jeg har vært glad i ro, men nå Jeg har ikke mulighet til å ha noen.
Truth: O, Pred: F

Eller hvorda

194it [00:04, 46.29it/s]

Det er viktig for meg at hver uke vi gjør noen gøy sammen, sånn at hun har gode minner fra sin barndom.
Truth: O, Pred: ORT

Andre ungdommer er interessert i sport, så medler de seg inn i idrettsklubbene.
Truth: O, Pred: M

Denne diskusjonen har ført til en polarisering, hvor på en side står motstandere, og de som støtter innføring av private skoler på den andre siden.
Truth: O, Pred: W

Hvis alle mennesker er tolerante overfor andre, alle sammen har det bra.
Truth: O, Pred: PUNC

Det er mye som vi kan gjøre for unge når er de organisert.
Truth: O, Pred: W

Spesielt i Norge, ble jeg fortalt, er det ikke mulig å finne nok kvalifiserte kandidater på alle høgskoler, selv om norske ungdommer er stadig sterkere interessert i høyere teoretisk utdanning.
Truth: O, Pred: M

Hva foregår på gata og hvordan kan og må vi oppføre oss på gata og i bilen.
Truth: O, Pred: F

Det største er at de får sjansen å få et barn, noe de har kanskje drømt om.
Truth: O, Pred: F

De nye forbilder er nå blitt pops

202it [00:04, 44.78it/s]

det var min mening.
Truth: CAP, Pred: PUNC

Jeg Håper at jeg klarer denne testen, og kanskje Bergenstesten, for å ha bedre sjanse til å komme inn.
Truth: CAP, Pred: R

Det er viktig at man finner på en aktivitet, for da får man mulighet til bli kjent med andre. Man kan alltid Lære noe positivt fra dem.
Truth: CAP, Pred: R

Et annet spørsmål er organdonasjon fra dyr. her er det nok mange som igjen vil se det som forkastelig å "blande" dyr med mennesker.
Truth: CAP, Pred: F

I mitt hjemland, Nederland, snakker de fleste engelsk, sånn at man ikke føler seg helt isolert i begynnelsen, men over lengre tid lønner det seg å snakke Nederlandsk for å komme seg med i det sosiale livet.
Truth: CAP, Pred: F

For eksempel kan de bruke tiden på etterutdanning, kunst, musikk osv. Når du er alene, kan folk slappe av og glemme stresset fra jobbet.
Truth: INFL, Pred: M

Det er én ting som vi kan unngå, forurensning av vannen.
Truth: INFL, Pred: R

Nå er det blitt vanskeligere for oss å akseptere at vi e

In [49]:
accuracy = sum([1 for t,p in zip(truths, preds) if t == p]) / len(truths)


0.18811881188118812


In [28]:
# do some predictions
model([dataset_test[0]["text"]])

tensor([4], device='mps:0')

In [ ]:
trainer.push_to_hub("tollefj/setfit-nocola-errortype-mps")